In [3]:
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"

In [4]:
# 데이터를 불러와 학습셋, 테스트셋으로 나눕니다.
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

# 단어의 수를 맞추어 줍니다. 
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

17464789/17464789 [==============================] - 2s 0us/step


In [5]:
X_train.shape

(25000, 500)

In [6]:
X_test.shape

(25000, 500)

In [7]:
model = models.Sequential()
model.add(layers.Embedding(5000, 100))
model.add(layers.Dropout(0.5))
model.add(layers.Conv1D(64, 5, padding='valid', activation='relu',strides=1)) #64*5*100+64 
model.add(layers.MaxPooling1D(pool_size=4)) #4개 중에서 max
#출력 차원은 입력 시퀀스의 길이를 pool_size로 나눈 값이 됩니다.
#MaxPooling1D 레이어는 다른 레이어와 함께 사용되며, 보통 Conv1D 레이어 다음에 사용됩니다. 이러한 방식으로, Conv1D 레이어를 통해 특징을 추출한 다음, 
# MaxPooling1D 레이어를 통해 중요한 특징을 추출하고 입력 데이터의 크기를 줄이는 것이 일반적인 CNN의 구성 방법
#https://tykimos.github.io/2017/08/17/Text_Input_Binary_Classification_Model_Recipe/
model.add(layers.LSTM(55)) #((64+55)*55+ 55)*4
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))
#model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         500000    
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 64)          32064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 55)                26400     
                                                                 
 dense (Dense)               (None, 1)                 56        
                                                        

In [8]:
# 모델의 실행 옵션을 정합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# 모델을 실행합니다.
history = model.fit(X_train, y_train, batch_size=40, epochs=100, validation_split=0.25, callbacks=[early_stopping])

Epoch 1/100
469/469 [==============================] - 49s 98ms/step - loss: 0.4152 - accuracy: 0.7897 - val_loss: 0.2767 - val_accuracy: 0.8829
Epoch 2/100
469/469 [==============================] - 44s 94ms/step - loss: 0.2369 - accuracy: 0.9054 - val_loss: 0.2665 - val_accuracy: 0.8867
Epoch 3/100
469/469 [==============================] - 44s 94ms/step - loss: 0.1828 - accuracy: 0.9311 - val_loss: 0.2696 - val_accuracy: 0.8928
Epoch 4/100
469/469 [==============================] - 48s 102ms/step - loss: 0.1459 - accuracy: 0.9481 - val_loss: 0.2778 - val_accuracy: 0.8946
Epoch 5/100
469/469 [==============================] - 48s 103ms/step - loss: 0.1191 - accuracy: 0.9571 - val_loss: 0.3228 - val_accuracy: 0.8896


In [9]:
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 13s 17ms/step - loss: 0.3534 - accuracy: 0.8806

 Test Accuracy: 0.8806
